In [50]:
#Load libraries
import os, torchvision, torch, glob, pathlib
import numpy as np
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable

In [51]:
#checking for calculator device
calculatorDevice=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [52]:
print(calculatorDevice)

cuda


In [53]:
#Transforms input data for processing
transformer=transforms.Compose([
    transforms.Resize((500, 500)),
    transforms.ToTensor(),  
    transforms.Normalize([0.5,0.5,0.5], 
                        [0.5,0.5,0.5])
])

In [54]:
#Data loader

#Path for training and testing data directory
trainingPath='data/id-fr-documents-zone-4/training_set'
testPath='data/id-fr-documents-zone-4/test_set'

trainingLoader=DataLoader(
    torchvision.datasets.ImageFolder(trainingPath,transform=transformer),
    batch_size=64, shuffle=True
)
testLoader=DataLoader(
    torchvision.datasets.ImageFolder(testPath,transform=transformer),
    batch_size=32, shuffle=True
)

In [55]:
# categories (classes) extraction
root=pathlib.Path(trainingPath)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

['authentics', 'counterfeits']


In [56]:
#Convolutional neuronal network (CNN) definition

class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #initial input shape= (256,3,150,150)
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #new shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        self.relu1=nn.ReLU()     
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #new shape= (256,12,75,75)
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #new shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #new shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()
        self.fc=nn.Linear(in_features=250 * 250 * 32,out_features=num_classes)
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)  
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output) 
            
        #Output shape like the last input shape (256,32,75,75)     
        output=output.view(-1,32*250*250)
                
        output=self.fc(output) 
        return output

In [57]:
classifierModel=ConvNet(num_classes=6).to(calculatorDevice)

In [58]:
#Optmizer and loss function
optimizer=Adam(classifierModel.parameters(),lr=0.001,weight_decay=0.0001)
lossFunction=nn.CrossEntropyLoss()
num_epochs=10

In [59]:
#calculating the size of training and testing images, .png images in input
trainingCount=len(glob.glob(trainingPath+'/**/*.png'))
testCount=len(glob.glob(testPath+'/**/*.png'))

In [60]:
print(trainingCount,testCount)

11 7


In [61]:
# Training and saving best performant model

bestAccuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    classifierModel.train()
    trainAccuracy=0.0
    trainLoss=0.0
    
    for i, (images,labels) in enumerate(trainingLoader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=classifierModel(images)
        loss=lossFunction(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        trainLoss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        trainAccuracy+=int(torch.sum(prediction==labels.data))
        
    trainAccuracy=trainAccuracy/trainingCount
    trainLoss=trainLoss/trainingCount
    
    
    # Evaluation on testing dataset
    classifierModel.eval()
    
    testAccuracy=0.0
    for i, (images,labels) in enumerate(testLoader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=classifierModel(images)
        _,prediction=torch.max(outputs.data,1)
        testAccuracy+=int(torch.sum(prediction==labels.data))
    
    testAccuracy=testAccuracy/testCount
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(trainLoss)+' Train Accuracy: '+str(trainAccuracy)+' Test Accuracy: '+str(testAccuracy))
    
    #Save the best model
    if testAccuracy>bestAccuracy:
        torch.save(classifierModel.state_dict(),'fake_ID_FR_zone_4_classifier.model') #Check the name according data input
        bestAccuracy=testAccuracy

Epoch: 0 Train Loss: tensor(1.9917) Train Accuracy: 0.0 Test Accuracy: 0.5714285714285714
Epoch: 1 Train Loss: tensor(0.) Train Accuracy: 1.0 Test Accuracy: 0.5714285714285714
Epoch: 2 Train Loss: tensor(0.) Train Accuracy: 1.0 Test Accuracy: 0.5714285714285714
Epoch: 3 Train Loss: tensor(0.) Train Accuracy: 1.0 Test Accuracy: 0.5714285714285714
Epoch: 4 Train Loss: tensor(0.) Train Accuracy: 1.0 Test Accuracy: 0.5714285714285714
Epoch: 5 Train Loss: tensor(0.) Train Accuracy: 1.0 Test Accuracy: 0.5714285714285714
Epoch: 6 Train Loss: tensor(0.) Train Accuracy: 1.0 Test Accuracy: 0.7142857142857143
Epoch: 7 Train Loss: tensor(0.) Train Accuracy: 1.0 Test Accuracy: 0.7142857142857143
Epoch: 8 Train Loss: tensor(0.) Train Accuracy: 1.0 Test Accuracy: 0.8571428571428571
Epoch: 9 Train Loss: tensor(0.) Train Accuracy: 1.0 Test Accuracy: 0.8571428571428571
